<h3>Tokenizer Evaluation on HAR, Sex 

Compare classification of magnitude model to tokenized magnitude with various quantisations. 

In [ ]:
# Configure GPU

import sys
sys.path.append('/Users/kyraedwards/Documents/Oxford/DPhil/Rotation_Wearables/Oxford_Wearables_Activity_Recognition/get_gpu.py')
from get_gpu import get_gpu
device = get_gpu()

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)
cudnn.benchmark = True


Set up

Load data

In [ ]:
#magnitude only


#tokenized



Train/ Test split

In [ ]:
# Hold out participants P101-P151 for testing (51 participants)
test_ids = [f'P{i}' for i in range(101,152)]
mask_test = np.isin(pid, test_ids)
mask_train = ~mask_test
X_train, Y_train, T_train, pid_train = \
    X[mask_train], Y[mask_train], T[mask_train], pid[mask_train]
X_test, Y_test, T_test, pid_test = \
    X[mask_test], Y[mask_test], T[mask_test], pid[mask_test]
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Magnitude, non tokenized model

In [ ]:
#

1 hour of data (non sleeping)

RNN - with arbitarly long input sequence. 